<a class="anchor" id="first-part"></a>

#!/usr/bin/env python<br># coding: utf-8

Author: Bao Cai

Course: Machine Learning for Descriptive Problems

Topic: NLP-Unsupervised

Start Date: 2020-03-11

Last Save: 2020-03-11

[1. Topic analysis](#topic-part)

There are quantitative measures for evaluating various aspects of clustering results and topic models intrinsically, and they can also be evaluated extrinsically by how well the clusters/topics serve some supervised task as features. In this exervise, however, we well fovus on qualitative evaluation of the results in terms of their descriptiveness. As in the example code, you may limit yourself to the 1000 first documents of ther corpus when performing clustering, in order to simplify the task and speed up experimentation, but use the whole corpus to calculate tf-idf features.

a. Experiment with different setups of the tf-idf feature extraction and clustering (k-means or hierarchical). In order to obtain meaningful results. When you arrive at a good configuration, describe it and motivate your chosen setup/parameters.

b. Inspect the keywords of the clusters. List the 10 first clusters out of all (o.e. not cherry picked examples) and privde an as descriptive label as possible for each of them.

c. Select one or two good clusters (that can be clearly interpreted) and one or two bad clusters (that might be difficult to interpret or distinguish). Motivate your choise (clusters may for instance, be overlapping, to broad/narrow or incoherent).

d. Repeat the experiment in (a) with LDA topic modelling instead (on the whole corpus), and explain briefly how the results compare to your previously chosen clustering setup. A few concrete examples may be helpful. Do your best to make sure the list of topic keywords are informative through appropriate post-processing.

[2. Word vectors](#word-vector)

a. Choose about 5 words (arbitrary) to use as seed words in the following experiment. Train word2vec vectors on the corpus while trying out variations on the parameters. Evaluate the vector models by inspecting the most similar words for each of the seed words, and try to identify qualitative differences between different parameter choices. Which parameters seem to have the most interesting effect? At what values? Motivate. Finally, study the qualitative effect of increasing the training data, by similarly comparing vectors trained with the best setup on the texts from the awards_2020 directory against vectors trained on the whole set of abstracts (1990-2002).

b. Repeat the experiment with ELMo from the lecture, with a different target word and diferent sentences. Choose a word that can have multiple senses, and construct 10 sentences that express 2-3 different senses of the word. Produce ELMo embeddings for the target word in each sentence and measure the similarity between the vectors. Evaluate in how many cases the measured similarities can be used to successfully distinguish between the different senses. Comment on the results, e.g. are you able to identify a particular way in which the model fails?

[top](#first-part)

In [41]:
import numpy
import os
import re
import binascii
import itertools
import heapq
import numpy as np
import pandas as pd
from time import time
from collections import Counter
from gensim import corpora, models
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [23]:
# Functions
def get_fnames(path='./Data'):
    """Read all text files in a folder.
    """
    fnames = []
    for root, _, files in os.walk(path):
        for fname in files:
            if fname[-4:] == '.txt':
                fnames.append(os.path.join(root, fname))
    return fnames


def read_file(fname):
    with open(fname, 'rt', encoding='latin-1') as f:
        # skip all lines until abstract
        for line in f:
            if "Abstract    :" in line:
                break

        # get abstract as a single string
        abstract = ' '.join([line[:-1].strip() for line in f])
        abstract = re.sub(' +', ' ', abstract)
        return abstract


def print_clusters(matrix, clusters, features, n_keywords=10):
    for cluster in range(min(clusters), max(clusters)+1):
        cluster_docs = [i for i, c in enumerate(clusters) if c == cluster]
        print("Cluster: %d (%d docs)" % (cluster, len(cluster_docs)))
        
        # Keep scores for top n terms
        new_matrix = np.zeros((len(cluster_docs), matrix.shape[1]))
        for cluster_i, doc_vec in enumerate(matrix[cluster_docs].toarray()):
            for idx, score in heapq.nlargest(n_keywords, enumerate(doc_vec), key=lambda x:x[1]):
                new_matrix[cluster_i][idx] = score

        # Aggregate scores for kept top terms
        keywords = heapq.nlargest(n_keywords, zip(new_matrix.sum(axis=0), features))
        print(', '.join([w for s,w in keywords]))
        print()


def vectorize_cluster(
    documents,
    sample_size=None,
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=0.0,
    max_features=None,
    use_idf=True,
    sublinear_tf=True,
    n_clusters=10,
    tol=0.0001
):
    vectorizer = TfidfVectorizer(
        ngram_range=ngram_range,
        analyzer=analyzer,
        max_df=max_df,
        min_df=min_df,
        max_features=max_features,
        use_idf=use_idf,
        sublinear_tf=sublinear_tf
    )
    token_matrix = vectorizer.fit_transform(documents)
    print(
        'The shape of the token matrix is:',
        token_matrix.toarray().shape
    )
    print()
    features = vectorizer.get_feature_names()
    print('The top 20 tokens are:')
    for feature, idf in sorted(
        zip(features, vectorizer._tfidf.idf_),
        key=lambda x:x[1]
    )[:20]:
        print('{:.2f}\t{}'.format(idf, feature))
    print()
    print('For the first 5 docs, the top tokens are:')
    for i in range(5):
        print('\nDocument {}, top terms by TF-IDF'.format(i))
        for feature, score in sorted(
            zip(features, token_matrix.toarray()[i]),
            key=lambda x: -x[1]
        )[:5]:
            print('{:.2f}\t{}'.format(score, feature))
    print()
    if not sample_size:
        matrix_to_cluster = token_matrix
    else:
        matrix_to_cluster = token_matrix[:sample_size]
    km = KMeans(
        n_clusters=n_clusters,
        tol=tol,
        random_state=44,
        verbose=False
    )
    km.fit(matrix_to_cluster)
    print_clusters(matrix_to_cluster, km.labels_, features)
    return (
        vectorizer,
        token_matrix,
        features,
        matrix_to_cluster,
        km
    )

In [22]:
%%time
documents = [read_file(file) for file in get_fnames('./Data/Arcada_BigDataAnalytic/')]

CPU times: user 17 µs, sys: 9 µs, total: 26 µs
Wall time: 25.7 µs



<a class="anchor" id="topic-part"></a>
### Topic analysis

[top](#first-part)

In [27]:
%%time
default_setup = vectorize_cluster(
    documents,
    sample_size=None,
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=0.0,
    max_features=None,
    use_idf=True,
    sublinear_tf=True,
    n_clusters=10,
    tol=0.0001
)

The shape of the token matrix is: (9923, 53816)

The top 20 tokens are:
1.03	the
1.03	of
1.03	and
1.04	to
1.05	in
1.13	this
1.14	for
1.18	is
1.19	will
1.26	be
1.29	on
1.31	with
1.33	that
1.40	are
1.40	research
1.41	by
1.43	as
1.51	from
1.55	an
1.61	these

For the first 5 docs, the top tokens are:

Document 0, top terms by TF-IDF
0.34	trafficking
0.22	drug
0.20	database
0.19	discovery
0.18	mislocalization

Document 1, top terms by TF-IDF
0.25	nmr
0.23	optically
0.20	ingap
0.18	hayes
0.17	inp

Document 2, top terms by TF-IDF
0.25	fabric
0.19	textiles
0.18	pesticides
0.18	weapons
0.18	drapes

Document 3, top terms by TF-IDF
0.33	thundersnow
0.19	rawinsonde
0.18	lightning
0.16	snow
0.15	forecasting

Document 4, top terms by TF-IDF
0.20	dots
0.19	tutorials
0.18	helium
0.17	condensates
0.17	quantum
Cluster: 0 (1505 docs)
polymer, magnetic, spin, nano, laser, films, quantum, optical, nanoscale, nanoparticles

Cluster: 1 (1494 docs)
species, birds, political, social, genetic, populations, evol

In [48]:
%%time
# So for this, I'll take out the top and bottom words
# Essentially stop words and
# words that are way too niche to be classified as descriptive
# By enforcing max_features=10000
# min_df is there to make sure of those 10000, there's no leftover
# Also looser clusters because too many things got put together
picky_setup = vectorize_cluster(
    documents,
    sample_size=None,
    ngram_range=(1, 1),
    analyzer='word',
    max_df=0.95,
    min_df=0.001,
    max_features=10000,
    use_idf=True,
    sublinear_tf=True,
    n_clusters=20,
    tol=0.0001
)

The shape of the token matrix is: (9923, 10000)

The top 20 tokens are:
1.13	this
1.14	for
1.18	is
1.19	will
1.26	be
1.29	on
1.31	with
1.33	that
1.40	are
1.40	research
1.41	by
1.43	as
1.51	from
1.55	an
1.61	these
1.61	project
1.61	at
1.84	have
1.88	which
1.92	new

For the first 5 docs, the top tokens are:

Document 0, top terms by TF-IDF
0.36	trafficking
0.24	drug
0.22	database
0.20	discovery
0.19	diseases

Document 1, top terms by TF-IDF
0.27	nmr
0.24	optically
0.15	wells
0.15	gaas
0.15	heterostructures

Document 2, top terms by TF-IDF
0.29	fabric
0.21	textiles
0.20	pesticides
0.20	weapons
0.16	protect

Document 3, top terms by TF-IDF
0.21	lightning
0.18	snow
0.17	forecasting
0.17	synoptic
0.17	dangerous

Document 4, top terms by TF-IDF
0.21	dots
0.20	tutorials
0.19	helium
0.18	quantum
0.17	path

Cluster: 0 (427 docs)
phase, fuel, sensor, market, polymer, optical, cell, devices, manufacturing, drug

Cluster: 1 (225 docs)
available, not, zooplankton, zoology, zones, zone, zonal, zno, z

In [51]:
%%time
# So for this, I'll take out the top and bottom words
# Essentially stop words and
# words that are way too niche to be classified as descriptive
# By enforcing max_features=10000
# min_df is there to make sure of those 10000, there's no leftover
# Also looser clusters because too many things got put together
pickier_setup = vectorize_cluster(
    documents,
    sample_size=None,
    ngram_range=(1, 1),
    analyzer='word',
    max_df=0.95,
    min_df=0.01,
    max_features=10000,
    use_idf=True,
    sublinear_tf=True,
    n_clusters=20,
    tol=0.0001
)

The shape of the token matrix is: (9923, 2378)

The top 20 tokens are:
1.13	this
1.14	for
1.18	is
1.19	will
1.26	be
1.29	on
1.31	with
1.33	that
1.40	are
1.40	research
1.41	by
1.43	as
1.51	from
1.55	an
1.61	these
1.61	project
1.61	at
1.84	have
1.88	which
1.92	new

For the first 5 docs, the top tokens are:

Document 0, top terms by TF-IDF
0.28	drug
0.26	database
0.24	discovery
0.23	diseases
0.21	protein

Document 1, top terms by TF-IDF
0.15	chemistry
0.15	nanostructures
0.15	associates
0.15	detected
0.15	academia

Document 2, top terms by TF-IDF
0.23	medical
0.18	testing
0.17	workers
0.17	could
0.17	personnel

Document 3, top terms by TF-IDF
0.18	observations
0.17	collection
0.17	vertical
0.16	weather
0.16	summer

Document 4, top terms by TF-IDF
0.22	quantum
0.22	path
0.21	nanostructures
0.18	semiconductor
0.17	integral

Cluster: 0 (398 docs)
chemistry, molecules, reactions, organic, metal, complexes, professor, compounds, reaction, chemical

Cluster: 1 (631 docs)
solar, stars, wind, mag

I tried with even stricter setup and it's a bit better, not too clear but I can identify topic much quicker

In [49]:
%%time
# So for this, it will be the same as above
# but with 2 words as well
two_words_setup = vectorize_cluster(
    documents,
    sample_size=None,
    ngram_range=(1, 2),
    analyzer='word',
    max_df=0.95,
    min_df=0.001,
    max_features=10000,
    use_idf=True,
    sublinear_tf=True,
    n_clusters=20,
    tol=0.0001
)

The shape of the token matrix is: (9923, 10000)

The top 20 tokens are:
1.13	this
1.14	for
1.18	is
1.19	will
1.22	of the
1.26	be
1.29	on
1.31	with
1.33	that
1.40	are
1.40	research
1.40	in the
1.41	by
1.43	as
1.51	from
1.55	an
1.57	will be
1.61	these
1.61	project
1.61	at

For the first 5 docs, the top tokens are:

Document 0, top terms by TF-IDF
0.19	drug
0.18	the database
0.17	database
0.16	discovery
0.15	diseases

Document 1, top terms by TF-IDF
0.20	nmr
0.13	and undergraduate
0.12	graduate and
0.11	gaas
0.11	heterostructures

Document 2, top terms by TF-IDF
0.28	fabric
0.16	protect
0.15	medical
0.15	military
0.13	polymerization

Document 3, top terms by TF-IDF
0.19	lightning
0.16	snow
0.15	forecasting
0.12	winter
0.12	cloud

Document 4, top terms by TF-IDF
0.17	dots
0.16	helium
0.14	quantum
0.14	path
0.14	statistical mechanics

Cluster: 0 (351 docs)
mantle, seismic, fault, deformation, magma, subduction, lithosphere, crust, the mantle, rocks

Cluster: 1 (774 docs)
galaxies, stars, st

This one is good too but at some point it's quite messed up so I'll go with the `pickier_setup`

In [62]:
%%time
new_vectorizer = TfidfVectorizer(
    documents,
    ngram_range=(1, 1),
    analyzer='word',
    max_df=0.95,
    min_df=0.01,
    max_features=10000,
    use_idf=True,
    sublinear_tf=True
)
# new_vectorizer = TfidfVectorizer()
word_tokenizer = new_vectorizer.build_tokenizer()
tokenized_text = [word_tokenizer(doc) for doc in documents]

dictionary = corpora.Dictionary(tokenized_text)
lda_corpus = [dictionary.doc2bow(text) for text in tokenized_text]
lda_model = models.LdaModel(lda_corpus, id2word=dictionary, num_topics=20)

for i, topic in lda_model.show_topics(num_topics=20, num_words=50, formatted=False):
    print("Topic", i)
    printed_terms = 0
    for term, score in topic:
        if printed_terms >= 10:
            break
        elif term in "this This that That these These have will Will\
        the of and to for in or The is be may an a with at are on by as from can \
        In it It has Has also not Not new".split():
            continue
        printed_terms += 1
        print("%.4f\t%s" % (score,term))
    print()

Topic 0
0.0058	reef
0.0044	coral
0.0044	corals
0.0036	rig
0.0035	preK
0.0030	WGBH
0.0028	PM
0.0025	reefs
0.0020	job
0.0012	judicial

Topic 1
0.0374	Available
0.0111	routing
0.0052	protocol
0.0050	query
0.0028	terminals
0.0025	adaptivity
0.0023	box
0.0022	RPI
0.0020	volatility
0.0019	Internet

Topic 2
0.0088	research
0.0035	project
0.0034	their
0.0027	workshop
0.0025	which
0.0024	study
0.0022	information
0.0022	development
0.0021	how
0.0021	data

Topic 3
0.0132	girls
0.0093	children
0.0083	science
0.0036	their
0.0034	project
0.0033	research
0.0033	students
0.0023	geoscience
0.0020	about
0.0019	fMRI

Topic 4
0.0090	data
0.0042	project
0.0028	such
0.0028	model
0.0026	which
0.0026	between
0.0026	research
0.0025	models
0.0025	how
0.0024	information

Topic 5
0.0088	theory
0.0058	problems
0.0053	study
0.0053	which
0.0049	systems
0.0048	equations
0.0043	project
0.0040	such
0.0034	geometry
0.0034	mathematical

Topic 6
0.0060	project
0.0041	Phase
0.0037	high
0.0034	DNA
0.0033	research
0.0030	dev

It's rather quick and output the results quite nicely. But with a little help of preprocessing on the tokenizer to filter out the noises, it's more meaningful than on its own

<a class="anchor" id="word-vector"></a>
### Word vectors

[top](#first-part)